In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

from google.cloud import bigquery
client = bigquery.Client()

sql_file_path = './sql/validator_timing_games.sql'
with open(sql_file_path, 'r') as file:
    query = file.read()

df = client.query(query).to_dataframe() 

In [ ]:
lime = '#caff00'
teal = '#00ff99'
green = '#00d37f'
navy = '#171c47'
dark_navy = '#171530'
title_color = green
label_color = green
chart_background_color = navy
chart_color = lime

# Find the adjusted minimum and maximum values for ms_diff
min_value = df['ms_diff'].min()
max_value = df['ms_diff'].max()
adjusted_min = min_value - (min_value % 25)
adjusted_max = max_value + (25 - max_value % 25)

# Create bin edges
bin_edges = np.arange(adjusted_min, adjusted_max + 25, 25)  # +25 to include the last edge
counts, _ = np.histogram(df['ms_diff'], bins=bin_edges)
max_count = counts.max()

plt.figure(figsize=(20, 12), facecolor=chart_background_color)
plt.axes(facecolor=chart_background_color)
plt.hist(df['ms_diff'], bins=bin_edges, color=chart_color, edgecolor=chart_background_color)
plt.axvline(0, color='tomato')
plt.title('distribution of winning block bid times | last 28 days', fontdict={'family': 'monospace', 'color' : green, 'size': 18}, pad=20)
plt.xlabel('ms diff from t-0', fontdict={'family': 'monospace', 'color' : label_color, 'size': 18}, labelpad=20)
plt.ylabel('count of slots', fontdict={'family': 'monospace', 'color' : label_color, 'size': 18}, labelpad=20)
plt.xlim(-2000, 4000)
plt.grid(True, color='white', linestyle='--', linewidth=0.5)
plt.xticks(fontsize=18, fontname='monospace', rotation=45, color=label_color)
y_ticks = range(0, max_count + 1000, 1000)
plt.yticks(y_ticks, fontsize=18, fontname='monospace', color=label_color)
plt.show()

![distribution of winning block bid times | last 28 days](assets/20231218-20240115/01_ms_diff_histogram_eden.png)

In [6]:
# Find the indices of the bins that are between 0 and 1000
bin_start = np.searchsorted(bin_edges, 0, side='right') - 1
bin_end = np.searchsorted(bin_edges, 1000, side='right')

# Create bins and counts for the range 0 to 1000
selected_bins = [(bin_edges[i], bin_edges[i+1]) for i in range(bin_start, bin_end - 1)]
selected_counts = counts[bin_start:bin_end - 1]
selected_histogram_table = pd.DataFrame({
    'Bin (Start, End)': selected_bins,
    'Count': selected_counts
})

sorted_histogram_table = selected_histogram_table.sort_values('Count', ascending=False)
sorted_histogram_table

,"Bin (Start, End)",Count
25,"(625, 650)",4924
24,"(600, 625)",4906
16,"(400, 425)",4874
26,"(650, 675)",4862
30,"(750, 775)",4724
13,"(325, 350)",4627
23,"(575, 600)",4591
12,"(300, 325)",4547
22,"(550, 575)",4348
15,"(375, 400)",4320


The above figure shows the distribution of relay timestamps for winning bids versus `block_timestamp` aka `t-0`. `t-0` is the point at which all transactions that were available to be included in the block, are evaluated. In theory any transactions arriving after `t-0`, whether mev or not, should not be included in the block. The above figure paints a different story; the distribution is weighted significantly past `t-0` with spikes around 400, 600 & 800 milliseconds. Interestingly there are also two spikes around 1100 and 2100 milliseconds.

In order to analyse further, we will look at how the bid value changes with respect to `t-0`. In other words, how much extra value is available to validators that delay n milliseconds past `t-0`?

In [ ]:
df['ms_diff'] = pd.to_numeric(df['ms_diff'], errors='coerce')
df['bid_delta_eth'] = pd.to_numeric(df['bid_delta_eth'], errors='coerce')

plt.figure(figsize=(20, 12), facecolor=chart_background_color)
plt.axes(facecolor=chart_background_color)
plt.scatter(df['ms_diff'], df['bid_delta_eth'], alpha=0.5, color=chart_color, edgecolor=chart_background_color)
plt.title('extra eth vs t-0 | last 28 days', fontdict={'family': 'monospace', 'color' : title_color, 'size': 18}, pad=20)
plt.xlabel('ms diff from t-0', fontdict={'family': 'monospace', 'color' : label_color, 'size': 18}, labelpad=20)
plt.ylabel('extra eth', fontdict={'family': 'monospace', 'color' : label_color, 'size': 18}, labelpad=20)
plt.axvline(0, color='tomato')
plt.xlim(-2000, 4000)
plt.ylim(-2, 6)
plt.xticks(fontsize=18, fontname='monospace', rotation=45, color=label_color)
plt.yticks(fontsize=18, fontname='monospace', color=label_color)
plt.grid(True)
plt.show()

![extra eth vs t-0 | last 28 days](assets/20231218-20240115/02_extra_eth_across_t_eden.png)

This figure shows the same bids data however this time we're looking at how much extra eth was gained versus the best bid available at `t-0`. The x-axis, as was the case in the previous figure, is the relay `timestamp` of the winning bid versus `t-0`. The y-axis is the difference in eth between the winning bid and the best bid at `t-0`. 

Clearly there is more eth to be gained by delaying. This is purely due to more transactions being available to builders/searchers as time passes. What this means for validators is that should they respect `t-0`, they are essentially leaving eth on the table. This is a clear incentive for validators to not respect `t-0`. The amount of eth avaiable to validators does not appear to be correlated with the delay size. 

Interestingly, some validators end up with less eth than the best bid at `t-0`. This may be due to cross-domain or off chain opportunities dissapearing. This is hard to analyse without more data.

Let's take a look at the validators that gained the most eth by not respecting `t-0`.

In [8]:
# Filter the DataFrame for ms_diff > 0 and bid_delta_eth > 0
filtered_df = df[(df['ms_diff'] > 0) & (df['bid_delta_eth'] > 0)]

# Sort the filtered DataFrame by bid_delta_eth in descending order
sorted_df = filtered_df.sort_values(by='bid_delta_eth', ascending=False)

result = sorted_df[['ms_diff', 'bid_delta_eth', 'block_number', 'proposer_pubkey', 'builder_pubkey', 'validator_operator']]

with pd.option_context('display.max_colwidth', None):
    display(result.head(10))

,ms_diff,bid_delta_eth,block_number,proposer_pubkey,builder_pubkey,validator_operator
167419,943,39.839748,18971364,0xb66e7c48fd1fa2bf3d2fb68454b80df302ffa150443bbf4b2a83a5d9651e15a7c37fc94bc6ccafeb718b74d7563e912b,0x8d2b4626cd64c2889ff4822c6bdb25161bbd8a3976f087086403c7ecf185eee4f2f71006c2a0dc8ade3bf0d8552236f4,lido - consensys codefi
159056,172,30.618301,18926740,0x94e96f02b3391da18bccf8b338fbba240ffbfe5a848b89d3f9cd566088815bd545a3f877287265ee06ed14d257ecceb8,0xb194b2b8ec91a71c18f8483825234679299d146495a08db3bf3fb955e1d85a5fca77e88de93a74f4e32320fc922d3027,lido - rocklogic gmbh
131574,488,14.596883,18991878,0xad8642a0eb203cbdeefafcd65af33a219feb99cbe53dbbaf1de0f969709721de9142808004ce7282bbe41cb4595b9291,0x945fc51bf63613257792926c9155d7ae32db73155dc13bdfe61cd476f1fd2297b66601e8721b723cef11e4e6682e9d87,coinbase - unknown
113835,1545,13.154018,18993061,0xa6ecbc9236335db6a1903e3ddaaffbd7e36483b7884810d1b40acc95a5199e40660d26f2fe9dc50acc39e249c034c3a4,0xae2ffc6986c9a368c5ad2d51f86db2031d780f6ac9b2348044dea4e3a75808b566c935099de8b1a1609db322f2110e7a,unknown - unknown
83431,441,11.883270,18926741,0xab6776bc5234bd73b7827aee978556e3ff8d07090756817f103f4dae5646be75dc093a48b3a5a03cc140f1bdb1616d58,0xacb407cfb554255db2fbbb320f79bb7f1cc1e8d2dc43324e8e31baafd0836340d49c43eebc51828f53bf6d364f9ac207,unknown - unknown
91675,1568,7.415314,18829775,0xb1edd7fc73f918b48cb33b0184dd663dcad6ce014ef1f36766eb9079ce0ca557ee7d72a924099aab604125a33871b19c,0x945fc51bf63613257792926c9155d7ae32db73155dc13bdfe61cd476f1fd2297b66601e8721b723cef11e4e6682e9d87,unknown - unknown
131187,777,6.646081,18836490,0xb7d3a212f7a02c104862da9d159d819c546602a616a8a35e7f980d9b169f08ecfdddebc1ed8aae2c44640bd184b11965,0x987ff80fcf6c5ee530f4a4352884cb89fc5f57ab287e58dd44d641f3bbe4cc40633d6ba0bbecc9c81b1d5be40a2abb99,coinbase - unknown
32091,1174,6.297879,18982359,0x8d7c141c95a9c45d55c9673cc1ceea5371ddb8b48e6758f9d6327ef92bec76c1802ee2454c33430aa33e74ed02a90e83,0xabcf636b7081e47edf77cf3f42b9e8b0c3a27a049b8eef3fa810ce6f7262036bea1910bc36a9a4791e8ee21587b82bf3,binance - unknown
145058,792,5.270907,18965781,0xaf7249107244c5b809240ba9a07170aaed927c07dd881366fb10b024d36b873cbff6a02588013030de7137d0721b72aa,0x8e96593b5418b162e7f7d5c165fef2566a67d5e03357baa780a2956a6f2c0b6426c20d7f82f527bcaa1719ebbc8d6048,stakefish - unknown
133849,395,4.610201,18828836,0x833134d4fdfb58dd9590228fbe6b8a741cba4d05c1e8b41c4224cc5fb6791e914c58ba2395d9ebb10d3d65012a41e7de,0xb9f88bc584a53fabb5b295e50638b20218c8b2303f3d26daef84ddb9609f1f93a95608806c94e564ebcd6e3976d3838d,lido - blockdaemon


Top spot in extra eth gained (for this time range) goes to `lido - consensus codefi` with ~40 ETH gained with a delay of ~1s after `t-0`. If we check block 18971364 in the [bids dashboard](https://data.edennetwork.io/block/18971364/bids) we can see that higher value bids start arriving around 500ms after `t-0`.

Now lets look at the validators that lost the most eth even though they selected bids that were delivered after `t-0`.

In [12]:
# Filter the DataFrame for ms_diff > 0 and bid_delta_eth < 0
filtered_df = df[(df['ms_diff'] > 0) & (df['bid_delta_eth'] < 0)]

# Sort the filtered DataFrame by bid_delta_eth in ascending order
sorted_df = filtered_df.sort_values(by='bid_delta_eth', ascending=True)


result = sorted_df[['ms_diff', 'bid_delta_eth', 'block_number', 'proposer_pubkey', 'builder_pubkey', 'validator_operator']]
with pd.option_context('display.max_colwidth', None):
    display(result.head(10))

,ms_diff,bid_delta_eth,block_number,proposer_pubkey,builder_pubkey,validator_operator
177243,28,-2.543716,19012724,0x9848b14ec65878ce90dd4bb57599916cd5fed736569dc5d0d2ee4073acb74d154291400a68531ca5e2fd8f68b4207812,0x945fc51bf63613257792926c9155d7ae32db73155dc13bdfe61cd476f1fd2297b66601e8721b723cef11e4e6682e9d87,lido - p2p.org - p2p validator
85518,131,-1.682419,18822826,0x8f2e582059533e2e0d96bf2a6eb57b6cfeaa2660fcc64b5f8a0faba4dc69450e368952473be76563f3ffb23bd928a3bb,0x8c0d45833884744d8cc4bb3438c3e93bcbc1436a25da5da44cc357e93bc0a7eea1d4664c287c4c63958eedada3846ae8,unknown - unknown
5858,731,-1.613603,18937973,0xb1a4a3ac1d86cf05748382dd7272e0f5dd04ad0fcfe2ae012f59fa7186b4621b0529a95a991127442ff3252d713522cf,0x978a35c39c41aadbe35ea29712bccffb117cc6ebcad4d86ea463d712af1dc80131d0c650dc29ba29ef27c881f43bd587,p2p.org - unknown
24100,487,-1.426023,18850166,0xb8440f5ab61f57adfcc95f03794331081666d52b3d6dd22015441f81e4664f580892cacc606169f947055bedd3ea74c9,0x94a076b27f294dc44b9fd44d8e2b063fb129bc85ed047da1cefb82d16e1a13e6b50de31a86f5b233d1e6bbaca3c69173,kraken - unknown
135487,1483,-1.394661,18979013,0x995ba28ad81f9950d04dd04936f010e8f5748515a02bf6c178262daa35e09270cf3127c5cd7c0e0133998f89b27b8e95,0xacfdcf458829f4693168a57d0659253069d687682bc64ec130d935ecb6e05ccfb80c138bd3cf53546c86715696612ec8,lido - bridgetower
155520,202,-1.358456,18848600,0xa9cb4149fae00a641952cf20847cffa471269b2340a7c95a7d52b7072c389baaeacd72cd19bf9338e201be9b431ef948,0xb194b2b8ec91a71c18f8483825234679299d146495a08db3bf3fb955e1d85a5fca77e88de93a74f4e32320fc922d3027,rocketpool - unknown
111987,481,-1.279007,18937694,0x94afe03f8392de38f316a2daf79705518958b75655fee4be1ab0304f896ee95418f7159ace6872914e96eca457064556,0xacb407cfb554255db2fbbb320f79bb7f1cc1e8d2dc43324e8e31baafd0836340d49c43eebc51828f53bf6d364f9ac207,unknown - unknown
172606,373,-1.152718,18829161,0x86fc19b9b5b2c14f9f3bd2d81489fb6f27739108eadd8262b29c4b70ed4c066815a8269f9b31ca313c3ae9f6f3aba628,0x94a076b27f294dc44b9fd44d8e2b063fb129bc85ed047da1cefb82d16e1a13e6b50de31a86f5b233d1e6bbaca3c69173,bitcoin suisse - unknown
23715,506,-0.958304,18867973,0xab94ab93c9a832311e5bec6c6ae399580120f283694e07ae9dc5142bd1bc1e599a85acc3d18527e0a8c806c6be4b8418,0x8c0d45833884744d8cc4bb3438c3e93bcbc1436a25da5da44cc357e93bc0a7eea1d4664c287c4c63958eedada3846ae8,kraken - unknown
120778,154,-0.870977,18886844,0xa7a60a563ced6130a7fc3b5fd1bf16420aadbf2beabf7ce81cedffefceb9b362b6dceee742c06b0f051567411efa531f,0x82ba7cadcdfc1b156ba2c48c1c627428ba917858e62c3a97d8f919510da23d0f11cf5db53cb92a5faf5de7d31bf38632,unknown - unknown


Top spot here goes to `lido - p2p.org`. The chose a bid that had ~2.5 less than the best bid available. This is confirmed in the [bids dashboard](https://data.edennetwork.io/block/19012724/bids).

Lets also delve into the validators that respected `t-0` but lost eth. These are likely due to validators selecting bids too early and better bids arriving between when they selected the bid and `t-0`.

In [13]:
# Filter the DataFrame for ms_diff <= 0 and bid_delta_eth < 0
filtered_df = df[(df['ms_diff'] <= 0) & (df['bid_delta_eth'] < 0)]

# Sort the filtered DataFrame by bid_delta_eth in descending order
sorted_df = filtered_df.sort_values(by='bid_delta_eth', ascending=True)

result = sorted_df[['ms_diff', 'bid_delta_eth', 'block_number', 'proposer_pubkey', 'builder_pubkey', 'validator_operator']]
with pd.option_context('display.max_colwidth', None):
    display(result.head(10))

,ms_diff,bid_delta_eth,block_number,proposer_pubkey,builder_pubkey,validator_operator
83402,-953,-2.139676,18929659,0xa6b5a99b686c30fa02b74642c63f1ebb248045ae9d107afbf86f11a499e6989c9c67e15394d52f17b6582980ea632f66,0xa2d994ea26d7c3b94b444fc2f2a6070d5cc8164e51d1a5c65db8ad6381ba5fc5e0eb3aa088b46e8da96b5167f9b97569,unknown - unknown
17733,-979,-1.612667,18884097,0x8afbe6e4a570af796a6d63f2efd23316095bc938b2bb328c3795dd8bd83f4236689765ea536a5c580aa86a03a88ccd57,0xa21a2f4807a2bcb6b07c10cea241322e0910c30869c1e4eda686b0d69bdcb74d2a140ef994afcf0bb38e0b960df4d2ee,lido - figment
23345,-1290,-1.600342,18927859,0xa62e9f504cc226a0c44ce46cdaacc92866c817894346202f2e8ab766b5e6c461e4821ef30e23a49b5276e4ae6f3a42e5,0xb67eaa5efcfa1d17319c344e1e5167811afbfe7922a2cf01c9a361f465597a5dc3a5472bd98843bac88d2541a78eab08,kraken - unknown
92338,-202,-1.296441,18829745,0x954c81a3deec7f02309be4fde67062364c7748b85eac1c9f8a5c5b34174a5f8793ed3cfe2a5507b8dd4e7b0ffafd7ab0,0xa4fb63c2ceeee73d1f1711fadf1c5357ac98cecb999d053be613f469a48f7416999a4da35dd60a7824478661399e6772,unknown - unknown
139856,-550,-1.288632,19006155,0x836937dfb286ba28138c5bfd6954a703672e2a39254ce5154b7605a612165846839d645468b0b8006f3ad4bf276ad76a,0xa21a2f4807a2bcb6b07c10cea241322e0910c30869c1e4eda686b0d69bdcb74d2a140ef994afcf0bb38e0b960df4d2ee,staked.us - unknown
139512,-814,-1.053303,18980018,0xa166a37d57e2ae1967131d40c937991a88acaa37b13b32271df5adba3d237a1bfe2be55d474cc2846b76006ed8a84c74,0xb9f88bc584a53fabb5b295e50638b20218c8b2303f3d26daef84ddb9609f1f93a95608806c94e564ebcd6e3976d3838d,staked.us - unknown
13307,-702,-1.048969,18865579,0x83af4ac03ada4c720fc673ed2bbabbcece70b51016c6d733aefb57b638bf930435c57624f73ebb6aa3bdb810c34db2e2,0xacfdcf458829f4693168a57d0659253069d687682bc64ec130d935ecb6e05ccfb80c138bd3cf53546c86715696612ec8,okx - unknown
42129,-427,-0.981911,18941736,0x95e1b046a68c2bf5d1844d991f0bff0faef33b47d16b4e71132b746dc5d6fda5b0f853417af520c46cfddf71749576b1,0x8e6df6e0a9ca3fd89db2aa2f3daf77722dc4fbcd15e285ed7d9560fdf07b7d69ba504add4cc12ac999b8094ff30ed06c,figment - unknown
169117,-491,-0.886090,19012751,0xb0986904123bad0cdd528c52ad945f3f6c299f30d912cb895229930f30c94444dce91120862acf72cb8f72c84bfe049f,0xa412007971217a42ca2ced9a90e7ca0ddfc922a1482ee6adf812c4a307e5fb7d6e668a7c86e53663ddd53c689aa3d350,lido - cryptomanufaktur
143411,-1972,-0.878215,18855414,0x95aaeb63d9e32af370fb3e13c4c97dd8746e1d73f581c7f09fb09ed96840133169989df685e6018f788dee1f71718863,0xacb407cfb554255db2fbbb320f79bb7f1cc1e8d2dc43324e8e31baafd0836340d49c43eebc51828f53bf6d364f9ac207,stakefish - unknown


Finally, to see all permutations we're just left with checking if any validators selected a bid earlier than `t-0` and ended up with more eth. We would assume not, but lets check.

In [14]:
# Filter the DataFrame for ms_diff > 0
filtered_df = df[(df['ms_diff'] < 0) & (df['bid_delta_eth'] > 0)]

# Sort the filtered DataFrame by bid_delta_eth in descending order
sorted_df = filtered_df.sort_values(by='bid_delta_eth', ascending=True)

result = sorted_df[['ms_diff', 'bid_delta_eth', 'block_number', 'proposer_pubkey', 'builder_pubkey', 'validator_operator']]
with pd.option_context('display.max_colwidth', None):
    display(result.head(10))

,ms_diff,bid_delta_eth,block_number,proposer_pubkey,builder_pubkey,validator_operator
96528,-90,0.015069,18838980,0xa4dcbb27d87fb2c0c1e160bc31e625920acba1409337767858dfa891fca9dc2312ce9afb4025faec47591c6d1723b30c,0xb783f81b2929e70c5b9a066d082d85dd12efa1c3e95185e18aabe6f4f93e387618532dc63ff80fdc3d9704f0ca5bc8c3,unknown - unknown


In conclusion, selecting the best bid at `t-0` is not the most profitable strategy for validators. Some validators chose to select the best bid before `t-0` and this was less optimal. 

This next section plots validator timings with respect to `t-0` as box plots. Only validators that mined >= 1000 blocks are included.

In [ ]:
from plotly.subplots import make_subplots

df['ms_diff'] = pd.to_numeric(df['ms_diff'], errors='coerce')

validator_counts = df['validator_operator'].value_counts().reset_index()
validator_counts.columns = ['validator_operator', 'count']

validator_counts = validator_counts[validator_counts['count'] >= 1000]

df_filtered = df.merge(validator_counts, on='validator_operator', how='inner')

df_filtered.sort_values(by='count', ascending=False, inplace=True)

fig = make_subplots(rows=1, cols=1, 
                    specs=[[{"secondary_y": True}]])

for operator in df_filtered['validator_operator'].unique():
    operator_data = df_filtered[df_filtered['validator_operator'] == operator]['ms_diff']
    fig.add_trace(go.Box(y=operator_data, name=operator, marker_color=chart_color), row=1, col=1)

for operator in validator_counts['validator_operator']:
    count = validator_counts[validator_counts['validator_operator'] == operator]['count'].values[0]
    fig.add_trace(go.Bar(x=[operator], y=[count], name=operator, opacity=0.3, marker_color=chart_color), row=1, col=1, secondary_y=True)

fig.update_layout(
    plot_bgcolor=chart_background_color,
    paper_bgcolor=chart_background_color,
    font=dict(size=18, color=title_color, family='Courier New, monospace'),    
    title=dict(text='millisecond difference box plot | mined blocks | mined blocks > 1000 | last 28 days', x=0.45, y=0.95, xanchor='center', yanchor='top'),
    yaxis=dict(
        title='ms diff from t-0',
        showgrid=True,
        gridcolor='rgba(255, 255, 255, 0.3)',
        zeroline=True,
        zerolinecolor='tomato',
        dtick=1000,
        color=label_color
    ),
    yaxis2=dict(title='blocks mined', side='right', showgrid=False, color=label_color),
    xaxis=dict(title='validator - operator', tickangle=45, color=label_color),
    width=2000,
    height=1200,
    showlegend=False        
)

fig.show()

![millisecond difference box plot | mined blocks | mined blocks > 1000 | last 28 days](assets/20231218-20240115/03_ms_diff_vs_mined_blocks_eden.png)

The above box plot displays all `validator | operator` pairs that mined at least 1000 blocks and how their blocks faired in comparison to `t-0` over the period. `validator | operator` are ordered (left to right) based on the number of blocks mined over the period. The first entry (`unknown - unknown`) is every block over the time range where we don't have a tag for both the `validator` and `operator` hence, it captures a wide range on entities and is pretty non conclusive in this analysis.

For all others entities what is clear is that on aggregate, most have their interquartile range slightly later than t-0. This may simply be due to geographical latency more than anything else. There are however, some interesting entities that appear to be maximising the timing games. The most exagerated ones are:

- `lido | skillz`
- `lido | attestant (bvi) limited`

The box plot intentifies others that also have interesting interquartile ranges. see:

- `binance | unknown`
- `lido | bridgetower`
- `lido | dsrv`
- `lido | p2p.org`

To dig into this a bit further we can order entities by how much extra revenue (ETH) they made versus the best bid available at `t-0`.

In [ ]:
bid_delta_eth_sum = df_filtered.groupby('validator_operator')['bid_delta_eth'].sum().reset_index()

fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])

for operator in df_filtered['validator_operator'].unique():
    operator_data = df_filtered[df_filtered['validator_operator'] == operator]['ms_diff']
    fig.add_trace(go.Box(y=operator_data, name=operator, marker_color=chart_color), row=1, col=1)

for index, row in bid_delta_eth_sum.iterrows():
    fig.add_trace(go.Bar(x=[row['validator_operator']], y=[row['bid_delta_eth']], name=row['validator_operator'], opacity=0.3, marker_color=chart_color, showlegend=False), row=1, col=1, secondary_y=True)

fig.update_layout(
    title=dict(text='millisecond difference | extra eth vs t-0 | mined blocks > 1000 | last 28 days', x=0.45, y=0.95, xanchor='center', yanchor='top'),
    plot_bgcolor=chart_background_color,
    paper_bgcolor=chart_background_color,
    font=dict(size=18, color=title_color, family='Courier New, monospace'),    
    yaxis=dict(
        title='ms diff from t-0',
        showgrid=True,
        gridcolor='rgba(255, 255, 255, 0.3)',
        zeroline=True,
        zerolinecolor='tomato',
        dtick=1000,
        color=label_color
    ),

    yaxis2=dict(title='extra eth vs t-0', side='right', showgrid=False, color=label_color, range=[0, 700]),
    xaxis=dict(title='validator - operator', tickangle=45, color=label_color),
    width=2000,
    height=1200,
    showlegend=False
)

fig.show()

![millisecond difference | extra eth vs t-0 | mined blocks > 1000 | last 28 days](assets/20231218-20240115/04_ms_diff_vs_eth_gained_eden.png)

The idea here was to see if the same entities are highlighted by the amount of extra revenue they gained. As can be seen from the graph above, `lido | skills` and `lido | attestant (bvi) limited` apear to have made noticeable gains. The amount of extra revenue they obtained versus `t-0` was as follows:

- `lido | skillz` -> 25 ETH
- `lido | attestant` -> 21 ETH

Interestingly, `lido - consensus codefi` made ~40 ETH extra and stick out substantially in the above figure. Their interquartile range is not as exegerated as the above mentioned two and they have a similar number of blocks. The explanation for this is they won block [block 18971364](https://data.edennetwork.io/block/18971364/bids) and the extra ETH available at ~1000ms after `t-0` was ~40 ETH (as shown previously). `lido - rocklogic gmbh` also stick out with their ~30 ETH [block 18926740](https://data.edennetwork.io/block/18926740/bids)

This graph keeps entities ordered by number of blocks mined, as defined in the previous chart, to allow for easy comparison. 